In [ ]:
import matplotlib.pyplot as plt
import os
import re
import string
import tensorflow as tf
import pandas as pd

from utils import predicted_test_data_to_result_csv
from keras import layers, losses, Input, Model
from keras.layers import Dense, Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.losses import sparse_categorical_crossentropy
from keras.metrics import sparse_categorical_accuracy
from keras.optimizers import Adam, SGD

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

In [ ]:
train_path = "data/base/goodreads_train.csv"
result_path = "data/base/goodreads_test.csv"

In [ ]:
df = pd.read_csv(train_path, sep=",")

Drop all lines with 0 in rating column

In [ ]:
index = df[(df['rating'] == 0)].index
df.drop(index, inplace=True)
df.reset_index(inplace=True, drop=True)

Targets DataFrames

In [ ]:
targets = df.pop('rating')
targets = targets - 1
# targets = tf.keras.utils.to_categorical(targets)

Features DataFrames

In [ ]:
features_names = ['review_text']
features = df[features_names]
tf.convert_to_tensor(features)

In [ ]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_spoilers = tf.strings.regex_replace(lowercase, '\*\* spoiler alert \*\*', ' ')
    return tf.strings.regex_replace(stripped_spoilers,
                                    '[%s]' % re.escape(string.punctuation),
                                    '')

In [ ]:
max_features = 5000  # Maximum vocab size.
sequence_length = 100

In [ ]:
vectorized_layer = tf.keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [ ]:
vectorized_layer.adapt(features)

In [ ]:
epochs = 20
model_nb = 1

embedding_dim = 50
learning_rate = 0.0001
batch_size = 20000
dropout_rate = 0.0

In [ ]:
input_text = Input(shape=(1,), dtype=tf.string)

vectorized_text = vectorized_layer(input_text)

embedding_layer = Embedding(max_features + 1, embedding_dim, input_length=sequence_length)(vectorized_text)

x = Conv1D(64, 3, activation = 'relu', padding = 'same')(embedding_layer)
x = Conv1D(64, 3, activation = 'relu', padding = 'same')(x)
x = MaxPooling1D(2)(x)

x = Conv1D(128, 5, activation = 'relu', padding = 'same')(x)
x = Conv1D(128, 5, activation = 'relu', padding = 'same')(x)
x = MaxPooling1D(3)(x)

x = Conv1D(256, 5, activation = 'relu', padding = 'same')(x)
x = Conv1D(256, 5, activation = 'relu', padding = 'same')(x)
x = MaxPooling1D(3)(x)

x = Conv1D(512, 5, activation = 'relu', padding = 'same')(x)
x = Conv1D(512, 5, activation = 'relu', padding = 'same')(x)
x = MaxPooling1D(3)(x)

global_max_pooling = GlobalMaxPooling1D()(x)

dense = Dense(32, activation='relu')(global_max_pooling)

output = Dense(5, activation='softmax')(dense)

cnn_model = Model(input_text, output)

cnn_model.summary()

In [ ]:
cnn_model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate=learning_rate),
                  metrics=sparse_categorical_accuracy)

In [ ]:
exp_name = f'cnn_model_{model_nb}_lr_{learning_rate}_bs_{batch_size}_dr_{dropout_rate}'

In [ ]:
cnn_model.fit(features,
              targets,
              validation_split=0.25,
              batch_size=batch_size,
              epochs=epochs,
              callbacks=[tf.keras.callbacks.TensorBoard("logs/cnn/" + exp_name)])